In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [11]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [19]:
## Defino funciones dimension mercado de trabajo e ingresos

# Componente: Inserción laboral

# T1. 
# Variable: Actividad productiva
# Indicador: Ratio M/V en tasa de empleo

def ratio_tasa_empleo(df, tipo='Aglomerado', base='Individual'):

    """
    INPUTS
    df: DataFrame. Tabla input EPH realizada con pool
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa de empleo M/V desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    # Check if the necessary columns exist in the input DataFrame
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe tener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")
        
    # Check if the input value of "tipo" is valid
    if tipo not in ['Aglomerado', 'Urbano']:
        return None, 'Error: tipo debe ser "Aglomerado" o "Urbano"'
    if base not in ['Individual', 'Hogar']:
        return None, 'Error: base debe ser "Individual" o "Hogar"'
    if not isinstance(df, pd.DataFrame):
        return None, 'Error: df debe ser un DataFrame de pandas'

    if tipo=='Aglomerado':
        var = 'AGLOMERADO' 
    elif tipo=='Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    numerador = df_temp[df_temp['ESTADO']==1].groupby(['CH04', var])['PONDERA_new'].sum()       # agrupo por genero y provincia y sumo
    
    size = df_temp[df_temp['ESTADO']==1].groupby(['CH04', var]).size().to_frame().unstack(level=0)
    size.columns = size.columns.get_level_values(1)

    denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum()
    df_tasa = numerador.div(denominador, fill_value=np.nan)
    df_tasa = df_tasa.to_frame().unstack(level=0)
    
    ratio = (df_tasa['PONDERA_new'][2]/df_tasa['PONDERA_new'][1] * 100).to_frame()
    ratio.rename(columns={0: 'Actividad productiva'}, inplace=True)
    
    ratio = ratio.join(size)
    ratio.rename(columns={1: 'N_v', 2: 'N_m'}, inplace=True)

    # Calculate error

    standard_error = np.sqrt((p1 * (1 - p1) / n1) + (p2 * (1 - p2) / n2)) / p2
    
    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    
    lower_bound = proportion_ratio - margin_of_error
    upper_bound = proportion_ratio + margin_of_error

    error = pd.DataFrame(index=ratio.index)

    return ratio, error

In [46]:
confidence_level = 0.95

In [37]:
var = 'PROVINCIA'
df = df_people_pool
df_temp = df.query('CH06 >= 16 & CH06 < 65')
numerador = df_temp[df_temp['ESTADO']==1].groupby(['CH04', var])['PONDERA_new'].sum().to_frame().unstack(level=0)       # agrupo por genero y provincia y sumo
denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum().to_frame().unstack(level=0)
df_tasa = numerador.div(denominador, fill_value=np.nan)
size = df_temp[df_temp['ESTADO']==1].groupby(['CH04', var]).size().to_frame().unstack(level=0)
size.columns = size.columns.get_level_values(1)

In [43]:
p1 = df_tasa['PONDERA_new'][2]
p2 = df_tasa['PONDERA_new'][1]
n1 = size[2]
n2 = size[1]

In [41]:
ratio = (p1/p2 * 100).to_frame()


In [55]:
standard_error = np.sqrt((p1 * (1 - p1) / n1) + (p2 * (1 - p2) / n2)) / p2

z = stats.norm.ppf((1 + confidence_level) / 2)
margin_of_error = (z * standard_error).to_frame()
    
lower_bound = ratio - margin_of_error
upper_bound = ratio + margin_of_error

In [60]:
pd.concat([lower_bound, ratio, upper_bound], axis=1)

,0,0,0
PROVINCIA,,,
2,90.088746,90.150144,90.211541
6,74.191021,74.219915,74.248810
10,75.348499,75.412363,75.476227
14,68.460849,68.499915,68.538980
18,66.586733,66.652274,66.717814
22,61.025642,61.093124,61.160606
26,75.701420,75.756036,75.810652
30,69.799480,69.850038,69.900597
34,50.176110,50.259593,50.343076


In [23]:
from scipy import stats


In [ ]:
import scipy.stats as stats

def calculate_proportion_ratio_confidence_interval(p1, n1, p2, n2, confidence_level):
    
    proportion_ratio = p1 / p2
    standard_error = np.sqrt((p1 * (1 - p1) / n1) + (p2 * (1 - p2) / n2)) / p2
    
    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    
    lower_bound = proportion_ratio - margin_of_error
    upper_bound = proportion_ratio + margin_of_error
    
    return lower_bound, upper_bound

In [20]:
ratio_empleo, error_empleo = ratio_tasa_empleo(df_people_pool, tipo='Urbano', base='Individual')


In [21]:
ratio_empleo

,Actividad productiva,N_v,N_m
PROVINCIA,,,
2,90.150144,642,594
6,74.219915,3985,3250
10,75.412363,1026,813
14,68.499915,1906,1487
18,66.652274,924,661
22,61.093124,829,608
26,75.756036,1176,973
30,69.850038,1496,1100
34,50.259593,841,459
